In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#from bqplot import pyplot as plt
#from bqplot import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
df = pd.read_csv('../data/cycles-with-features.csv', header=0).drop(labels='Unnamed: 0', axis=1)
df = df[df.Cycle >= 1].reset_index().drop('index', axis=1)

### Increasing/decreasing-load trainng set


Helper methods for isolating heating/cooling subsets of cycles

In [3]:
def inc(cycle_df):
    start = cycle_df[cycle_df.Min==1].Time.values[0]
    end = cycle_df[cycle_df.Max==1].Time.values[0]
    return cycle_df[(cycle_df.Time > start) & (cycle_df.Time < end)]

def dec(cycle_df):
    start = cycle_df[cycle_df.Max==1].Time.values[0]
    end = cycle_df.iloc[-1:].Time.values[0]
    return cycle_df[(cycle_df.Time > start) & (cycle_df.Time < end)]

In [15]:
df_inc = pd.DataFrame()
df_dec = pd.DataFrame()
for cycle in np.arange(1, 47):
    cycle_df = df[df.Cycle==cycle].reset_index()    
    df_inc = df_inc.append(inc(cycle_df), ignore_index=True)
    df_dec = df_dec.append(dec(cycle_df), ignore_index=True)

# Automation via _GridSearchCV_

In [14]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GroupShuffleSplit, GridSearchCV, TimeSeriesSplit
np.warnings.filterwarnings('ignore')

In [13]:
features = ['Time', 'Load', 'Cycle', 'Area', 'Heating', 'Cooling', 'HCprop', 'Period', 'Tail', 'Belly', 'Kurt', 'Skew']
ests = [LinearRegression(), Ridge(), Lasso(), ElasticNet()]
param_grid = {
    'alpha': np.linspace(0.01, 10.0, 30)
    #'warm_start': [True, False]
}
scores = {}
best_est = {}
tscv = TimeSeriesSplit(n_splits=5)

# HOTFIX
X=df[df.Min==1].drop(['Datetime'], axis=1).fillna(0)

for feature in features:
    cv = None
    for i, est in enumerate(ests):
        #t_features = features[:]; t_features.remove(feature)
        t_features = ['Time', 'Cycle']
        y = X[feature]
        
        # run gridsearch cross-validation
        if i == 0:
            cv = GridSearchCV(est, {}, cv=tscv, refit=True).fit(X[t_features], y)
        else:
            cv = GridSearchCV(est, param_grid, cv=tscv, refit=True).fit(X[t_features], y)
        score = cv.score(X[t_features], y)
        
        # enumerate scores by features
        if feature in scores.keys():
            scores[feature].append(score)
        else:
            scores[feature] = [score]
            
        # store the best estimator for curve generation later
        if feature not in best_est.keys():
            best_est[feature] = (None, 0) # default score
        elif score > best_est[feature][1]: # best score
            best_est[feature] = (cv.best_estimator_, score)

    #best_est[feature] = (ests[scores[feature].index(max(scores[feature]))],\
    #                     max(scores[feature])) #(cv.best_estimator_, cv.best_score_, scores[feature])

/home/nwchen/.local/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/nwchen/.local/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/nwchen/.local/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/nwchen/.local/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You 